In [ ]:
# https://m.weibo.cn/api/container/getIndex?containerid=232478__6798536848_-_profile_selected
import requests
import json
import re
import urllib3
import sqlite3 #数据库操作
import time
import random

In [11]:
def init_db(dbpath):
    sql = '''
        CREATE TABLE joinStopic
        (
            id INTEGER  PRIMARY KEY AUTOINCREMENT,
            user_id varchar,
            Stopic_name varchar
        )
    '''
    conn = sqlite3.connect(dbpath) #文件存在即打开，不存在即创建
    cursor = conn.cursor()
    cursor.execute(sql)
    conn.commit()
    conn.close()

In [12]:
def saveData2DB(datalist,dbpath):
    # init_db(dbpath)
    conn = sqlite3.connect(dbpath)
    cursor = conn.cursor()
    sql = "insert into joinStopic(user_id,Stopic_name) values(?,?)"
    for data in datalist:
        cursor.execute(sql,data)
        conn.commit()
    cursor.close()
    conn.close()

In [68]:
def get_Stopic(userList, base_url):
    for userID in userList:
        count = 0
        isok = ""
        url = base_url + userID + "_-_profile_selected"
        # print(url)
        try:  
            urllib3.disable_warnings()
            web_data = requests.get(url, headers=headers,verify=False)    #F12查看data信息
            js_con = web_data.json()    #转换一下数据格式
            count += 1
            #print(js_con)
            #isok = js_con['ok']

            headers['cookie'] = cookie_list[count%2] #换cookie
            cards_group = js_con['data']['cards'][0]['card_group']
            cards_data_list = []
            for card in cards_group:    #依次循环获得cards
                cards_data = []
                Stopic_name = card["title_sub"]     #超话名字

                cards_data.append(userID)
                cards_data.append(Stopic_name)
                
                cards_data_list.append(cards_data)
                print(userID + " " + Stopic_name) #显示爬取到第几条

            saveData2DB(cards_data_list,dbpath)
            time.sleep(random.uniform(2,7))
        except Exception as e:
            print("出错了" ,e)
            time.sleep(60*5)
            continue

In [69]:
def select_users(dbpath):
    con = sqlite3.connect(dbpath)
    cur = con.cursor()
    arr = []
    sql = "select user_id from users"
    cur.execute(sql)
    row = cur.fetchone()
    while row:
        arr.append(row[0])
        row = cur.fetchone()
    return arr

In [72]:
base_url = "https://m.weibo.cn/api/container/getIndex?containerid=232478__"
dbpath = "J_SNOWMAN_4709336482318969.db"
# init_db(dbpath)
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9;',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7,ko;q=0.6',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'cookie':'WEIBOCN_FROM=1110006030; SCF=AmHexnmgkvxyMfOl7teccNNtI3I-aIptcc95t0_nN667Ax4MI-HcYc4Yn9rVZfwi7R5iVxR1cvE_3g-0S_Hyz4k.; SUB=_2A25MtmWvDeRhGeNH6lcV-SbLyzSIHXVsWQvnrDV6PUJbktAKLXnMkW1NSsw9B3oAwogWIs1su3XjfP2YaqPpC8F9; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WhnV5Gg861gnb3kKqBEUXZD5NHD95Qf1K2fSh.RS05RWs4DqcjGxNHjIgSaTBtt; SSOLoginState=1639060991; _T_WM=39966695760; MLOGIN=1; XSRF-TOKEN=d39146; M_WEIBOCN_PARAMS=luicode%3D20000174%26uicode%3D20000174',
    'Referer': 'https://m.weibo.cn',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36',
    'Host': 'm.weibo.cn'
}

cookie_list = ['_T_WM=35998642978; XSRF-TOKEN=2a5d46; WEIBOCN_FROM=1110006030; SCF=AmHexnmgkvxyMfOl7teccNNtI3I-aIptcc95t0_nN667eIMQraixhRTsjvgg8Qaib0ZjQVgjmomek7-IMzmuxME.; SUB=_2A25MtxHpDeRhGeNH6lcV-SbLyzSIHXVsW7-hrDV6PUJbktCOLXTykW1NSsw9B0RSzs3ybqybrB1Ry7o50p_CqQUO; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WhnV5Gg861gnb3kKqBEUXZD5NHD95Qf1K2fSh.RS05RWs4DqcjGxNHjIgSaTBtt; SSOLoginState=1639145913; MLOGIN=1; M_WEIBOCN_PARAMS=luicode%3D10000011%26lfid%3D102803%26uicode%3D20000174'
              ,'_T_WM=35998642978; XSRF-TOKEN=2a5d46; WEIBOCN_FROM=1110006030; SUB=_2A25MtxFtDeRhGeBM6VcX8C3EyTSIHXVsW78lrDV6PUJbkdAKLRTGkW1NRPviX3eN0QRbcyVoFVkMv2yAsc48YSCx; SCF=AvfUwvC7H1yX3lSizYoZGICM8vTX7Rw7uItS2MtmSt_QS-kHqbteIm1zDrCbNp1N8IPCnfTeTsORs92q8tQpoQA.; SSOLoginState=1639145789; MLOGIN=1; M_WEIBOCN_PARAMS=luicode%3D10000011%26lfid%3D102803%26uicode%3D20000174']
#userList = select_users(dbpath)
userList = ['6501248567',
'7641535385',
'1991436775',
'7718205116',
'5050283658',
'7233520838',
'3209499247',
'7465208951',
'1887857693',
'3966277613',
'2606829143',
'3525444054',
'6348082433',
'5846583136']
get_Stopic(userList, base_url)

6501248567 mmrl
6501248567 SnowMan组合
6501248567 村上真都raul
6501248567 柯哀
6501248567 rlmm
6501248567 快新
6501248567 目黑莲
6501248567 崔雪莉
6501248567 摩尔庄园
6501248567 罗云熙
7641535385 莲理枝memi
7641535385 SnowMan组合
7641535385 目黑莲
7641535385 道枝駿佑
7641535385 热情杨溢
7641535385 迪丽热巴
7641535385 洋迪
7641535385 电视剧安乐传
7641535385 剧版你是我的荣耀官方
7641535385 剧版你是我的荣耀
1991436775 余翔
1991436775 舞者张翰
1991436775 龚俊
1991436775 石倚洁
1991436775 声入人心
1991436775 羽生田挙武
1991436775 ARASHI
7718205116 莲理枝memi
7718205116 目黑莲
7718205116 道枝駿佑
7718205116 SnowMan组合
5050283658 莲理枝memi
5050283658 SnowMan组合
5050283658 目黑莲
5050283658 想谈一场偶像剧般的恋爱
5050283658 博君一肖
7233520838 目黑莲
7233520838 SnowMan组合
7233520838 消失的初恋
3209499247 莲理枝memi
3209499247 目黑莲
3209499247 SnowMan组合
3209499247 张艺兴
3209499247 浪花男子
3209499247 莲理枝mechi
3209499247 道枝駿佑
3209499247 黄潇
3209499247 白敬亭
3209499247 Bboy乔治
7465208951 目黑莲
7465208951 SnowMan组合
7465208951 FF14
7465208951 宮脇咲良
7465208951 全职高手
7465208951 最终幻想14
7465208951 少年歌行
1887857693 村上真都raul
1887857693 SnowMan组合
18878

In [ ]:
# 缺了6501248567
# 7641535385
# 1991436775
# 7718205116
# 5050283658
# 7233520838
# 3209499247
# 7465208951
# 1887857693
# 3966277613
# 2606829143
# 3525444054
# 6348082433
# 5846583136